In [ ]:
import numpy as np
import pandas as pd

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import glob

import os

from PIL import Image

In [ ]:
train_root = "/Users/nicklittlefield/Downloads/unifesp-x-ray-body-part-classifier/train/train/"
train_csv = pd.read_csv("train.csv")

outdir = "/Users/nicklittlefield/Downloads/unifesp-x-ray-body-part-classifier/train_png"
if not os.path.exists(outdir):
    os.mkdir(outdir)


In [ ]:
train_csv

In [ ]:
def read_xray(path, out_pth, fix_monochrome=True):
    dicom = pydicom.read_file(os.path.join(root_dir, file_name))
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    img_xray = Image.fromarray(data)
    img_sample.save(out_pth + path.split("/")[-1].replace(".dcm", ".png"))
    
    return out_pth

In [ ]:
for i, row in train_csv.iterrows():
    img_sop = row.SOPInstanceUID
    img_pth = list(glob.iglob(train_root + "**/**/**/*" + img_sop + "*.dcm"))[0]
    read_xray(img_pth, outdir)